In [1]:
import numpy as np
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from icecream import ic
from tqdm import trange
from numba import njit

%load_ext snakeviz
# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [2]:
%pip install pyvlfeat

  Using cached pyvlfeat-0.1.1a3.tar.gz (159 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pyvlfeat
Failed to build pyvlfeat
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyvlfeat
ERROR: Could not build wheels for pyvlfeat, which is required to install pyproject.toml-based projects


In [3]:
# from cyvlfeat import sift 
# frames,desc=sift.sift(img,compute_descriptor=True,n_levels=1)
# ip=(frames.T)[0:2,:]
# desc=desc.astype(np.float)

In [4]:
# %pip install cyvlfeat

In [15]:
def compute_sift_points(img1, img2, marg):
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    # # img1 = cv2.medianBlur(img1, ksize = 5)
    # img1 = cv2.normalize(img1, None, 0, 255, cv2.NORM_MINMAX)
    # # img1 = img1.astype(np.float32)

    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    # # img2 = cv2.medianBlur(img2, ksize = 5)
    # img2 = cv2.normalize(img2, None, 0, 255, cv2.NORM_MINMAX)
    # # img2 = img2.astype(np.float32)


    # sift = cv2.SIFT_create(int nfeatures=0, int nOctaveLayers=3, double contrastThreshold=0.04, double edgeThreshold=10, double sigma=1.6)
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # # orb = cv2.ORB_create()
    # # kp1, des1 = orb.detectAndCompute(img1, None)
    # # kp2, des2 = orb.detectAndCompute(img2, None)
    
    # FLANN_INDEX_KDTREE = 1
    # index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    # search_params = dict(checks=50)   # or pass empty dictionary

    # flann = cv2.FlannBasedMatcher(index_params, search_params)
    # matches = flann.knnMatch(des1, des2, k=2)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    print('Number of matches:', np.size(matches,0))

    good_matches = []
    for m, n in matches:
        if m.distance < marg*n.distance:
            good_matches.append([m])

    draw_params = dict(matchColor=(255,0,255), singlePointColor=(0,255,0), matchesMask=None, flags=cv2.DrawMatchesFlags_DEFAULT)
    img_match = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches, None, **draw_params)

    x1 = np.stack([kp1[match[0].queryIdx].pt for match in good_matches],1)
    x2 = np.stack([kp2[match[0].trainIdx].pt for match in good_matches],1)

    x1 = cv.homogenize(x1, multi=True)
    x2 = cv.homogenize(x2, multi=True)

    print('Number of good matches:', np.size(x1,1))

    return x1, x2, img_match

def get_sift_plot_points(img1_pts, img2_pts, img1):
    x = [img1_pts[0,:], np.size(img1,1)+img2_pts[0,:]]
    y = [img1_pts[1,:], img2_pts[1,:]]
    return x, y

def plot_sift_points(x1, x2, img1, img_match, path, save=False):
    x, y = get_sift_plot_points(x1, x2, img1)

    fig = plt.figure(figsize=(18,9))
    ax = plt.axes()

    ax.plot(x, y, 'o-', ms=5, lw=1, color='magenta')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_aspect('equal')
    # ax.legend(loc="upper right")
    ax.imshow(cv2.cvtColor(img_match, cv2.COLOR_BGR2RGB))
    fig.tight_layout()
    if save:
        fig.savefig(path, dpi=300)
    plt.show()

In [6]:
from skimage import data
from skimage import transform
from skimage.color import rgb2gray
from skimage.feature import match_descriptors, plot_matches, SIFT

def compute_sift_points_sk(img1, img2, marg):

    # img1 = rgb2gray(img1)
    # img2 = rgb2gray(img2)
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    descriptor_extractor = SIFT()

    descriptor_extractor.detect_and_extract(img1)
    keypoints1 = descriptor_extractor.keypoints
    descriptors1 = descriptor_extractor.descriptors

    descriptor_extractor.detect_and_extract(img2)
    keypoints2 = descriptor_extractor.keypoints
    descriptors2 = descriptor_extractor.descriptors

    good_matches = match_descriptors(descriptors1, descriptors2, max_ratio=marg, cross_check=True)
    fig, ax = plt.axes()
    print('Number of matches:', np.size(good_matches,0))

    # plt.gray()

    # plot_matches(ax, img1, img2, keypoints1, keypoints2, good_matches)
    
    x1 = np.stack([keypoints1[match[0]].pt for match in good_matches],1)
    x2 = np.stack([keypoints2[match[1]].pt for match in good_matches],1)

    print(x1.shape, x2.shape)

    x1 = cv.homogenize(x1, multi=True)
    x2 = cv.homogenize(x2, multi=True)
    print(x1.shape, x2.shape)
    print('Number of good matches:', np.size(x1,1))


    return x1, x2

In [7]:
def compute_ransac_iterations(alpha, epsilon, s):
    T = np.ceil(np.log(1-alpha) / np.log(1-epsilon**s))
    return T

In [8]:
def estimate_E_robust(K, x1_norm, x2_norm, n_its, n_samples, err_threshold_px):
    
    err_threshold = err_threshold_px / K[0,0]
    best_inliers = None
    best_E = None
    max_inliers = 0

    for t in trange(n_its):

        rand_mask = np.random.choice(np.size(x1_norm,1), n_samples, replace=False)
        E = cv.estimate_E_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], enforce=True, verbose=False)

        D1, D2 = cv.compute_epipolar_errors(E, x1_norm, x2_norm)
        inliers = ((D1**2 + D2**2) / 2) < err_threshold**2

        n_inliers = np.sum(inliers)

        if n_inliers > max_inliers:
            best_inliers = np.copy(inliers)
            best_E = np.copy(E)
            max_inliers = n_inliers

            print(np.sum(inliers), end='\r')
        
    return best_E, best_inliers

In [9]:
def plot_cameras_and_3D_points(X, C_arr, axis_arr, s, path, save=False):
    
    fig = plt.figure(figsize=(8,6))
    ax = plt.axes(projection='3d')

    ax.plot(X[0], X[1], X[2], '.', ms=2, color='magenta', label='3D points')
    cv.plot_cameras_and_axes(ax, C_arr, axis_arr, s)

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.axis('equal')
    ax.view_init(elev=-50, azim=-104, roll=45)
    plt.legend(loc="lower right")
    if save:
        fig.savefig(path, dpi=300)
    plt.show()

In [10]:
path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-4'
data = r'\data-2023\data'
compex = r'\compEx2data.mat'
img1 = r'\fountain1.png'
img2 = r'\fountain2.png'
# img1 = r'\round_church1.jpg'
# img2 = r'\round_church2.jpg'
report = r'\report-images'
fountain1 = cv.load_image(path+data+img1)
fountain2 = cv.load_image(path+data+img2)
K = cv.convert_mat_to_np(path+data+compex, 'K')
K_inv = np.linalg.inv(K)

ransac = True
plt_3D = True
save = False
snakeviz = False

In [17]:
ind = 3
marg = 0.8
x1, x2, fountain_match = compute_sift_points(fountain1, fountain2, marg)
# plot_sift_points(x1, x2, fountain1, fountain_match, path+report+'/CE2_sift_{}.png'.format(ind), save=True)

x1_norm = cv.transform_and_dehomogenize(K_inv, x1)
x2_norm = cv.transform_and_dehomogenize(K_inv, x2)

np.save(path+data+'/CE2_x1_norm_{}.npy'.format(ind), x1_norm)
np.save(path+data+'/CE2_x2_norm_{}.npy'.format(ind), x2_norm)

Number of matches: 4381
Number of good matches: 353


In [12]:

x1_norm = cv.transform_and_dehomogenize(K_inv, x1)
x2_norm = cv.transform_and_dehomogenize(K_inv, x2)

# ind = 4
# marg = 0.7
# x1, x2 = compute_sift_points_sk(fountain1, fountain2, marg)

# x1_norm = cv.transform_and_dehomogenize(K_inv, x1)
# x2_norm = cv.transform_and_dehomogenize(K_inv, x2)

# np.save(path+data+'/CE2_x1_norm_{}.npy'.format(ind), x1_norm)
# np.save(path+data+'/CE2_x2_norm_{}.npy'.format(ind), x2_norm)

In [9]:
ind = 2
x1 = cv.homogenize(cv.convert_mat_to_np(path+data+'/CE2_sift_matlab.mat', 'xA'), multi=True)
x2 = cv.homogenize(cv.convert_mat_to_np(path+data+'/CE2_sift_matlab.mat', 'xB'), multi=True)

x1_norm = cv.transform_and_dehomogenize(K_inv, x1)
x2_norm = cv.transform_and_dehomogenize(K_inv, x2)

np.save(path+data+'/CE2_x1_norm_{}.npy'.format(ind), x1_norm)
np.save(path+data+'/CE2_x2_norm_{}.npy'.format(ind), x2_norm)

In [18]:
ind = 3
x1_norm = np.load(path+data+'/CE2_x1_norm_{}.npy'.format(ind))
x2_norm = np.load(path+data+'/CE2_x2_norm_{}.npy'.format(ind))

if False:
    x = cv.convert_mat_to_np(path+data+'/compEx3data.mat', 'x')
    x1 = cv.dehomogenize(x[0,0])
    x2 = cv.dehomogenize(x[0,1])
    x1_norm = cv.dehomogenize(np.linalg.inv(K) @ x1)
    x2_norm = cv.dehomogenize(np.linalg.inv(K) @ x2)

if ransac:
    n_its = 100000
    n_samples = 8
    err_threshold_px = 4
    E, inliers = estimate_E_robust(K, x1_norm, x2_norm, n_its, n_samples, err_threshold_px)

    np.save(path+data+'/CE2_E_robust_{}.npy'.format(ind), E)
    np.save(path+data+'/CE2_inliers_{}.npy'.format(ind), inliers)
    print('No. inliers:', np.sum(inliers))
    
if snakeviz:
    %snakeviz estimate_E_robust(K, x1_norm, x2_norm)

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 93/100000 [00:00<02:00, 827.83it/s]

  1%|          | 669/100000 [00:00<01:27, 1139.25it/s]

  2%|▏         | 2039/100000 [00:02<01:35, 1029.47it/s]

 12%|█▏        | 12320/100000 [00:14<01:50, 795.21it/s]

 52%|█████▏    | 52174/100000 [00:51<00:37, 1261.34it/s]

100%|██████████| 100000/100000 [01:53<00:00, 881.39it/s]

No. inliers: 235


In [27]:
ind = 2
E = np.load(path+data+'/CE2_E_robust_{}.npy'.format(ind))
inliers = np.load(path+data+'/CE2_inliers_{}.npy'.format(ind))
x1_norm = np.load(path+data+'/CE2_x1_norm_{}.npy'.format(ind))
x2_norm = np.load(path+data+'/CE2_x2_norm_{}.npy'.format(ind))

x1_norm = x1_norm[:,inliers]
x2_norm = x2_norm[:,inliers]

F = cv.convert_E_to_F(E, K, K)

print('\nE:', E)
print('F:', F)

P2_arr = cv.extract_P_from_E(E)
P1 = cv.get_canonical_camera()

X_arr_dh = np.array([cv.dehomogenize(cv.triangulate_3D_point_DLT(P1, P2, x1_norm, x2_norm, print_svd=False)) for P2 in P2_arr])
P2_valid, X_valid = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr_dh)
np.save(path+data+'/CE2_X_valid_{}.npy'.format(ind), X_valid)
np.save(path+data+'/CE2_P2_valid_{}.npy'.format(ind), P2_valid)

i = 0
X_valid = X_arr_dh[i]
P2_valid = P2_arr[i]

if plt_3D:
    P_arr = np.array([P1, P2_valid])
    C_arr, axis_arr = cv.compute_camera_and_normalized_principal_axis(P_arr, multi=True)
    s = 1
    plot_cameras_and_3D_points(X_valid, C_arr, axis_arr, s, path+report+'/CE2_3D_P2_{}_{}.png'.format(i+1, ind), save)


E: [[ -0.04140149   6.0290843   -0.69372926]
 [  9.40405325   0.77104285  14.4009753 ]
 [ -0.44186359 -16.08240252   1.        ]]
F: [[-1.00207849e-09  1.45680418e-07 -1.91482995e-04]
 [ 2.27229599e-07  1.85991205e-08  5.95944531e-04]
 [-2.56765358e-04 -1.31258941e-03  1.00000000e+00]]
[1.72204901e+01 1.72204901e+01 1.36453547e-16]


No. valid coords for each camera pair: [   0 3680 1840 1840]
Argmax(P2_arr): 1
[[-0.          0.83732562]
 [-0.         -0.05681539]
 [-0.         -0.54374426]
 [ 1.          1.        ]] [[0.         0.80569901]
 [0.         0.01468276]
 [1.         0.59214316]
 [1.         1.        ]]
